<a href="https://colab.research.google.com/github/tristantoupin/COMP551-KaggleCompetition/blob/master/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
path_to_data = "../data/train_images.npy"
path_to_labels = "../data/train_labels.csv"
path_to_test = "../data/test_images.npy"

In [3]:
def get_data(path_x, path_y):
    data = np.load(path_x, encoding = 'bytes')
    labels_df = pd.read_csv(path_y)
    labels_df.Category = pd.Categorical(labels_df.Category)
    y = labels_df.Category.cat.codes.values
    X = list(data[:, 1])
    return X, y

In [5]:
X, y = get_data(path_to_data, path_to_labels)
print("Shape of training dataset:", len(X))
X_test = list(np.load(path_to_test, encoding = 'bytes')[:, 1])
print("Shape of testing dataset:", len(X_test))
unique_classes = set(y)
print("Number of classes:", len(unique_classes))

Shape of training dataset: 10000
Shape of testing dataset: 10000
Number of classes: 31


In [6]:
# parameters
learning_rates = [1e-4, 1e-3, 1e-2, 1e-1]
nodes = [50, 100, 1000, 10000, 12000]
hidden_layers = [1, 2, 3, 4]
batch_size = [1/100, 1/10, 1]
activation = ['sigmoid', 'tanh', 'relu']

In [7]:
# X = np.array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
# y = np.array([[0, 1, 1, 0]]).T
# unique_classes = (np.unique(y))

In [56]:
class NN():
    def __init__(self, learning_rate, layers, nodes_per_layer, b_size, activation, patience, random_state = 1234):
        np.random.seed(random_state)
        n = [nodes_per_layer for i in range(layers)] + [len(unique_classes)]   # Model architecture
        
        # init params
        self.weights = []
        self.random_state = random_state
        for p, _ in enumerate(n[:-1]):
            self.weights.append(2 * np.random.random(tuple(n[p:p + 2])) - 1)        
        self.batch_size = b_size
        self.activation_fct = activation
        self.patience = patience
        self.learning_rate = learning_rate
        self.accuracies = []


    def sigmoid(self, x, deriv = False):
        if(deriv==True):
            return x*(1-x)
        print(type(1/(1+np.exp(-x))))
        return 1/(1+np.exp(-x))
        
        
    def predict(self, inputs, test = False):
        layers_ = [inputs]
        for w in self.weights:
            layers_.append(self.sigmoid(np.dot(layers_[-1], w)))
        if test:   # if its testing data, return only output of model
            return np.argmin(layers_[-1], axis = 1).reshape(len(layers_[-1]), -1)
        return layers_
    
    
    def update_weights(self, outputs, layers_):
        error = outputs - layers_[-1]
        delta = error * self.sigmoid(layers_[-1], deriv = True)
        
        for c, (l, w) in enumerate(zip((layers_[1:-1][::-1]), (self.weights[::-1]))):
            adjustment = layers_[len(self.weights) - c - 1].T.dot(delta) * self.learning_rate
            self.weights[len(self.weights) - c - 1] += adjustment
            error = delta.dot(w.T)
            delta = error * self.sigmoid(l, deriv=True)
        
        adjustment = layers_[0].T.dot(delta) * self.learning_rate
        self.weights[0] += adjustment

        return error, delta
            
    
    def train(self, xs_all, ys_all):
        epoch = 0
        while (True):
            epoch += 1

            xs, x_val, ys, y_val = train_test_split(xs_all, ys_all, test_size=0.90, random_state = self.random_state)
            
            # Predict            
            layers = self.predict(xs)
            
            # Get accuracy of model
            predictions = np.argmin(layers[-1], axis = 1).reshape(len(layers[-1]), -1)
            self.accuracies.append(accuracy_score(ys, predictions))
            
            # Update Weights
            ys_one_hot = np.array(np.eye(len(unique_classes))[ys])
            l, w = self.update_weights(ys_one_hot, layers)                

            if not(epoch % 2):
                print("Epoch #" + str(epoch))
                print("Training accuracy:", self.accuracies[-1])
                preds_val = self.predict(x_val, test = True)
                acc = accuracy_score(y_val, preds_val)
                print("Test accuracy:", acc)
                if acc < self.accuracies[-1] * 0.9:
                    return




In [57]:
nn = NN(1e-2, 2, 10000, 256, 'sigmoid', 3)

In [ ]:
nn.train(X, y)

[25 20 23 ..., 21  3 13]
